# Logarithmic mean divisia index (LMDI) application on European Union carbon emissions from 2000 to 2017

$$\text { Kaya identity : } C = P \times \frac{G}{P} \times \frac{E}{G} \times \frac{C}{E} = P \times Gp \times EI \times CI $$


$$ \text { P : the population,}$$
$$ \text { G : the GDP,}$$
$$ \text { E : the consumption of fossil energy,}$$  
$$ \text { C : } CO_{2} \text { emission due to fossil energy,}$$
$$ \text { Gp : the GDP per capita,}$$
$$ \text { EI : the energy intensity,}$$
$$ \text { CI : the carbon intensity.}$$

$$ \text{Additive decomposition Change scheme LMDI formulas : }$$

\begin{aligned}
&\Delta C_{t o t}=C^{T}-C^{0}=\Delta C_{\text {P}}+\Delta C_{Gp}+\Delta C_{EI}+\Delta C_{\text {CI}} \\
&\Delta C_{P}=\frac{C^{T}-C^{0}}{\ln C^{T}-\ln C^{0}} \ln \left(\frac{P^{T}}{P^{0}}\right)\\
&\Delta C_{Gp}=\frac{C^{T}-C^{0}}{\ln C^{T}-\ln C^{0}} \ln \left(\frac{Gp^{T}}{Gp^{0}}\right)\\
&\Delta C_{EI}=\frac{C^{T}-C^{0}}{\ln C^{T}-\ln C^{0}} \ln \left(\frac{EI^{T}}{EI^{0}}\right)\\
&\Delta C_{CI}=\frac{C^{T}-C^{0}}{\ln C^{T}-\ln C^{0}} \ln \left(\frac{CI^{T}}{CI^{0}}\right)\\
\end{aligned}

### Variables

In [1]:
import pandas as pd
data_1 = pd.read_csv("/home/cj/Bureau/MILESTONE_6/european_dataset_1.csv")
data_2 = pd.read_csv("/home/cj/Bureau/MILESTONE_6/european_dataset_2.csv")
pop = data_1['pop']
emi = data_1['fossil_emi']
gdp_cap = data_2['gdp_cap']
ei = data_2['ei_fossil'] # energy intensity of fossil energies (coal, oil, gas)
ci = data_2['ci_fossil'] # carbon intensity of fossil energies (coal, oil, gas)

var_list = [pop, gdp_cap, ei, ci]

### Indice 0 and T (T>0) correspond respectively to the base year and the target year. 4 intervals of time are taken into account.
    1 : [2000 - 2004]
    2 : [2005 - 2009]
    3 : [2010 - 2013]
    4 : [2014 - 2017]
    5 : [2000 - 2017]

In [2]:
#intervals
inter_1 = [10, 14] 
inter_2 = [15, 19]
inter_3 = [20, 23]
inter_4 = [24, 27]
inter_5 = [10, 27]

In [3]:
import numpy as np

def from_list_to_pandas(e, p):
    df = pd.DataFrame([e, p], columns = ['pop effect', 'gdp cap effect', 'energy intensity effect',
                                        'carbon intensity effect', 'total effect'])
    df = df.transpose()
    df.columns=['Effect on CO2 emissions [Mtoe]', 'Percentage']
    return df

"""
This function gives the effect of a variable on the variation of the CO2 emissions.
"""
def rate_effect(emi, var):
    return ((emi[1]-emi[0])/(np.log(emi[1])-np.log(emi[0])))*np.log(var[1]/var[0])

"""
Retreive the value of a variable and the carbon emissions for a corresponding interval of time.
"""
def get_target_base_value(interval, emission, variable):
    emi = list(); var = list()
    for ind in interval:
        emi.append(emission[ind])
        var.append(variable[ind])
    return emi, var

"""
This function retreives the effect, on the variation of the CO2 emissions, of each variables
pointed as responsible of these emissions in European Union.
"""
def get_lmdi(var_list, emi, interval):
    list_effect = list()
    for i in range(len(var_list)):
        emi_, var_ = get_target_base_value(interval, emi, var_list[i])
        list_effect.append(rate_effect(emi_, var_))
    
    list_effect.append(sum(list_effect))
    list_percentage_effect = list()
    for effect in list_effect:
    
        list_percentage_effect.append((effect/np.abs(list_effect[len(list_effect)-1]))*100)
        
    x = np.abs(list_percentage_effect[len(list_effect)-1])
    list_percentage_effect[len(list_effect)-1] = x
    df = from_list_to_pandas(list_effect, list_percentage_effect)
    
    return df

In [4]:
lmdi_2 = get_lmdi(var_list, emi, inter_2)
lmdi_2

,Effect on CO2 emissions [Mtoe],Percentage
pop effect,49.032036,13.449505
gdp cap effect,575.327658,157.812580
energy intensity effect,-1020.641631,-279.962360
carbon intensity effect,31.718062,8.700276
total effect,-364.563876,100.000000


In [5]:
lmdi_1 = get_lmdi(var_list, emi, inter_1)
lmdi_1

,Effect on CO2 emissions [Mtoe],Percentage
pop effect,32.065043,16.099347
gdp cap effect,1646.812268,826.838177
energy intensity effect,-1501.792392,-754.026010
carbon intensity effect,22.084920,11.088486
total effect,199.169839,100.000000


In [6]:
lmdi_3 = get_lmdi(var_list, emi, inter_3)
lmdi_3

,Effect on CO2 emissions [Mtoe],Percentage
pop effect,3.718399,1.108363
gdp cap effect,203.997015,60.806460
energy intensity effect,-505.958294,-150.813643
carbon intensity effect,-37.242879,-11.101180
total effect,-335.485759,100.000000


In [7]:
lmdi_4 = get_lmdi(var_list, emi, inter_4)
lmdi_4

,Effect on CO2 emissions [Mtoe],Percentage
pop effect,21.249500,9.503285
gdp cap effect,-251.708914,-112.570253
energy intensity effect,357.760227,159.998939
carbon intensity effect,96.300812,43.068029
total effect,223.601624,100.000000


In [8]:
lmdi_5 = get_lmdi(var_list, emi, inter_5)
lmdi_5

,Effect on CO2 emissions [Mtoe],Percentage
pop effect,131.262931,47.765666
gdp cap effect,2163.642138,787.334299
energy intensity effect,-2739.308085,-996.815126
carbon intensity effect,169.596985,61.715161
total effect,-274.806031,100.000000


In [9]:
"""
In order to vizualize the effect of the varibles on carbon emissions, the pourcentage obtained with
the function get_lmdi are grouped in function of the different variables. 
"""
def lmdi_variables(lmdi_list, dict_, names, column):
    for name in names:
        temp = list()
        for lmdi in lmdi_list:
            temp.append(lmdi[column][name])
        dict_[name] = temp
    return pd.DataFrame.from_dict(dict_)

names = ['pop effect', 'gdp cap effect', 'energy intensity effect', 'carbon intensity effect', 'total effect']
dict_ = { i : [] for i in names }
lmdi_list = [lmdi_1, lmdi_2, lmdi_3, lmdi_4, lmdi_5]

lmdi_var = lmdi_variables(lmdi_list, dict_, names, 'Percentage')
lmdi_var

,pop effect,gdp cap effect,energy intensity effect,carbon intensity effect,total effect
0,16.099347,826.838177,-754.026010,11.088486,100.0
1,13.449505,157.812580,-279.962360,8.700276,100.0
2,1.108363,60.806460,-150.813643,-11.101180,100.0
3,9.503285,-112.570253,159.998939,43.068029,100.0
4,47.765666,787.334299,-996.815126,61.715161,100.0


## VISUALIZATION

In [13]:
import plotly.graph_objects as go

months = ['2000-2004', '2005-2009', '2010-2013', '2014-2017', '2000-2017']

fig = go.Figure()

fig.add_trace(go.Bar(x=months, y=lmdi_var['pop effect'], name='Population effect',
                     marker_color='indianred'))
fig.add_trace(go.Bar(x=months, y=lmdi_var['gdp cap effect'], name='Gdp per cap effect',
                     marker_color='lightsalmon'))
fig.add_trace(go.Bar(x=months, y=lmdi_var['energy intensity effect'], name='Energy intensity effect',
                     marker_color='Red'))
fig.add_trace(go.Bar(x=months, y=lmdi_var['carbon intensity effect'], name='Carbon intensity effect',
                     marker_color='Blue'))
fig.add_trace(go.Bar(x=months, y=lmdi_var['total effect'], name='Total effect',
                     marker_color='Green'))

fig.update_yaxes(title_text="Rate effect %")
fig.update_layout(barmode='group', xaxis_tickangle=-45,
                  title ="Decomposition of the changes in CO2 emissions in European Union : 2000–2017")
fig.show()